## Ola

### Set all paths

In [1]:
import os
from datasets import load_from_disk
from pathlib import Path

project_root = os.path.abspath(os.path.join(Path.cwd(), "..", ".."))
corpus_config= "LitSearch_corpus_clean"
query_config= "LitSearch_query"
dataset_dir = os.path.join(project_root, "dataset")
run_dir = os.path.join(project_root, "run_files")
stopwords_path = os.path.join(dataset_dir, "stopwords.txt")
corpus_path = os.path.join(dataset_dir, corpus_config)
query_path = os.path.join(dataset_dir, query_config)


print(f"Project root directory: {project_root}")
print(f"Dataset directory: {dataset_dir}")
print(f"Run files directory: {run_dir}")
print(f"Corpus path: {corpus_path}")
print(f"Query path: {query_path}")

import sys
sys.path.append(project_root)


Project root directory: /home/akash/UNH/CS853_IR/Project/irg_final_project
Dataset directory: /home/akash/UNH/CS853_IR/Project/irg_final_project/dataset
Run files directory: /home/akash/UNH/CS853_IR/Project/irg_final_project/run_files
Corpus path: /home/akash/UNH/CS853_IR/Project/irg_final_project/dataset/LitSearch_corpus_clean
Query path: /home/akash/UNH/CS853_IR/Project/irg_final_project/dataset/LitSearch_query


### Load dataset

In [2]:
corpus_full = load_from_disk(corpus_path)['full']
queries_full = load_from_disk(query_path)['full']
print(f"Corpus details: {corpus_full}")
print(f"Queries details: {queries_full}")

queries = [q['query'] for q in queries_full]
print(f"Number of queries loaded: {len(queries)}")

Corpus details: Dataset({
    features: ['corpusid', 'title', 'abstract', 'citations', 'full_paper'],
    num_rows: 64183
})
Queries details: Dataset({
    features: ['query_set', 'query', 'specificity', 'quality', 'corpusids'],
    num_rows: 597
})
Number of queries loaded: 597


### Query Expansion (LLM)

In [3]:
expansion_path = os.path.join(dataset_dir, "expansions_all_req.jsonl")

# if expansions_all_req.jsonl already exists, load it directly
if os.path.exists(expansion_path):
    print("expansions_thesaurus.jsonl already exists.")
    import json
    expanded_queries = []
    with open(expansion_path, "r", encoding="utf8") as f:
        for line in f:
            item = json.loads(line)
            expanded_queries.append(item["expanded_query"])
    print(f"Loaded {len(expanded_queries)} expanded queries from expansions_all_req.jsonl")

# Ensure we have plain text strings for expanded queries
expanded_query_texts = [
    item["expanded_query"] if isinstance(item, dict) else item
    for item in expanded_queries
]

expansions_thesaurus.jsonl already exists.
Loaded 597 expanded queries from expansions_all_req.jsonl


### LLM Reranking

In [4]:
from src.classes.llm_reranker import LLMReranker

initial_run_path = os.path.join(run_dir, "dense_expanded_100.run") # Run file after LLM query expansion and bm25
output_run_path = os.path.join(run_dir, "BM25_LLMExp_DenseRetrieval_LLMRe-rank.run")  # append _LLM-Rerank before .run
corpus_path = os.path.join(project_root, "data", "corpus_jsonl", "corpus.jsonl") # path to corpus in jsonl format
log_path = os.path.join(project_root, "logs", "BM25_LLMExp_DenseRetrieval_LLMRe-rank.jsonl") # logs llm reranker activity


reranker = LLMReranker()  # uses OLLAMA_MODEL or defaults to "llama3.1:8b-instruct-q8_0-16k"
reranker.rerank_runfile(
    initial_run_path= initial_run_path, 
    corpus_path     = corpus_path,
    output_run_path = output_run_path,
    queries         = expanded_query_texts,  # query list
    log_path        = log_path,
    top_k=20,
    run_tag="BM25_LLMExp_DenseRetrieval_LLMRe-rank"
)

Using Ollama model: llama3.1:8b-instruct-q8_0-16k


LLM reranking: 100%|██████████| 596/596 [1:17:56<00:00,  7.85s/it]


## Evaluation

In [7]:
!cd ../.. && \
python3 evaluation/evaluate.py \
  --qrels evaluation/litsearch.qrel \
  --metric ndcg@50 \
  --output evaluation/results/BM25_LLMExp_DenseRetrieval_LLMRe-rank \
  --runs run_files/BM25_LLMExp_DenseRetrieval_LLMRe-rank.run

Summary (ndcg@50)
- BM25_LLMExp_DenseRetrieval_LLMRe-rank: mean=0.380100, stderr=0.014960
Summary file written to: evaluation/results/BM25_LLMExp_DenseRetrieval_LLMRe-rank/summary_ndcg@50.csv


In [6]:
!cd ../.. && \
python3 evaluation/evaluate.py \
  --qrels evaluation/litsearch.qrel \
  --metric map \
  --output evaluation/results/BM25_LLMExp_DenseRetrieval_LLMRe-rank\
  --runs run_files/BM25_LLMExp_DenseRetrieval_LLMRe-rank.run

Summary (map)
- BM25_LLMExp_DenseRetrieval_LLMRe-rank: mean=0.305753, stderr=0.014883
Summary file written to: evaluation/results/BM25_LLMExp_DenseRetrieval_LLMRe-rank/summary_map.csv
